<a href="https://colab.research.google.com/github/SwayamInSync/prompt-expansion/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from huggingface_hub import notebook_login

notebook_login()

# Loading Language Model Gemma

In [ ]:
# loading base model (not instruction tuned)

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

In [25]:
# loading the instruction tuned model

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer_it = AutoTokenizer.from_pretrained("rootacess/gemma-instruction-tuned")
model_it = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
def generate(input_text, model, tokenizer):
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

  outputs = model.generate(**input_ids, max_new_tokens=128)
  return tokenizer.decode(outputs[0])

In [29]:
prompt = """
You are an intelligent assistant which can analyze the given instruction and expand its detail to make it more clear, understandable and through the point.
Instruction: {instruction}

Given above instruction, please expand the instruction by adding more details to it, avoid directly answering the instruction and only give the new detailed instruction back
"""

instruction = "Write a poem on cows"

input_text = prompt.format(instruction=instruction)
print(input_text)


You are an intelligent assistant which can analyze the given instruction and expand its detail to make it more clear, understandable and through the point.
Instruction: Write a poem on cows

Given above instruction, please expand the instruction by adding more details to it, avoid directly answering the instruction and only give the new detailed instruction back



# Testing the base model (poor performance)

> Can be better if use Large language models (gemma-7B)

In [34]:
print(generate(input_text, model, tokenizer))

<bos>
You are an intelligent assistant which can analyze the given instruction and expand its detail to make it more clear, understandable and through the point.
Instruction: Write a poem on cows

Given above instruction, please expand the instruction by adding more details to it, avoid directly answering the instruction and only give the new detailed instruction back
<eos>


# Testing the instruction tuned model (better performance)

> Can be even increase on using Large language models (gemma-7B)

In [30]:
print(generate(input_text, model_it, tokenizer_it))

<bos>
You are an intelligent assistant which can analyze the given instruction and expand its detail to make it more clear, understandable and through the point.
Instruction: Write a poem on cows

Given above instruction, please expand the instruction by adding more details to it, avoid directly answering the instruction and only give the new detailed instruction back
Sure, here is the expanded instruction:

Write a poem about cows, describing their physical characteristics, behavior, and the role they play in the agricultural industry.<eos>
